
**Team Members:**
*  Nour Osama **202201874** section: 01
<br>
*  Layla Zayed **202201906** section: 02






# Libraries and Setup

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from snowballstemmer import stemmer

In [ ]:
!pip install Arabic-Stopwords
import arabicstopwords.arabicstopwords as stp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.5 MB/s eta 0:00:00


In [ ]:
import emoji

In [ ]:
!pip install python-terrier

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.6 MB/s eta 0:00:00
  Created wheel for chest: filename=chest-0.2.3-py3-none-any.whl size=7611 sha256=fc0001a94fde44f86136d1ecfcbd048842c9ffcfd1

In [ ]:
import pyterrier as pt

if not pt.java.started():
    pt.java.init()

terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


# Preprocessing

**Stop Words Removal**

In [ ]:
def remove_stop_words(sentence):
    terms=[]
    stopWords= set(stp.stopwords_list())
    for term in sentence.split() :
        if term not in stopWords :
           terms.append(term)
    return " ".join(terms)

**Regix**

In [ ]:
def remove_special_char(text):
    text = re.sub(r"http\S+|www\S+", " ", text)
    text = re.sub(r"@\S+", " ", text)
    text = re.sub(r"[\,+\#_\|\:\?\?\/\=\!+\.+]", " ", text)
    text = re.sub(r'["\'<>_\-|\\;~»«]', " ", text)
    text = re.sub(r'\s+', " ", text).strip()
    return(text)

**Normalization**

In [ ]:
def normalize(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ئ", "ء", text)

    return(text)

**Emojis Removal**

In [ ]:
def remove_emojis(text):
  return emoji.replace_emoji(text, replace='')

**Stemming**

In [ ]:
ar_stemmer = stemmer("arabic")
def stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])

**Applay Preprocessing**

In [ ]:
def preprocess(sentence):
  sentence = remove_stop_words(sentence)
  sentence = remove_special_char(sentence)
  sentence = remove_emojis(sentence)
  sentence = normalize(sentence)
  sentence = stem(sentence)
  return sentence

**Collection Preparation**

In [ ]:
tweets = pd.read_csv('/content/tweets.csv', sep=',')

In [ ]:
tweets['processed_tweets'] = tweets['tweetText'].apply(preprocess)
tweets

,tweetID,tweetText,processed_tweets
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر...",اعدام لعامل مطعم قتل زميل طعنا بيادر ايد محكم ...
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب ا...,اخبار تاجيل محا 7 ارهابي سبب غياب دفاع اجل محك...
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذ...,عشان اعطي وحد صميم صرت ترم اعذار حقق عالميهه ا...
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مف...,نهد ثما قتل تفجير انتحار سيار مفخخ معمل غاز ري...
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر...,بحر ضبط مطلوب متورط تفجير عكر شرق بقي موضوع اضغط
...,...,...,...
4995,551574113324920832,«بوكو حرام »تخطف 40 شابا في شمال شرق نيجيريا -...,وكو حرام تخطف 40 شاب نيجيري رياض رياض وكو حرام...
4996,551574871797272576,قصيدة مهداة إلى الشيخ/ ضيف الله بن سمار كلمات/...,قصيد مهد شيخ ضيف الله سمار كلما سلطان حامد سما...
4997,551574876092256256,العفو عند المقبرة!!,عفو مقبره
4998,551575878141222912,#للمرة الأولى منذ 92 عاما.. تركيا تسمح ببناء ك...,مره اول 92 عام تركي تسمح بناء كنيس جديد مجل رساءل


In [ ]:
tweets["docno"] = tweets["tweetID"].astype(str)
tweets

,tweetID,tweetText,processed_tweets,docno
0,549679192804061184,"الاعدام لعامل مطعم قتل زميله طعناً في ""البيادر...",اعدام لعامل مطعم قتل زميل طعنا بيادر ايد محكم ...,549679192804061184
1,549699343666532352,#الأخبار ▪ تأجيل محاكمة 7 إرهابيين بسبب غياب ا...,اخبار تاجيل محا 7 ارهابي سبب غياب دفاع اجل محك...,549699343666532352
2,549711593487888387,@helale9999 عشآن أعطيتك وحده صميم صرت ترمي أعذ...,عشان اعطي وحد صميم صرت ترم اعذار حقق عالميهه ا...,549711593487888387
3,549719610459967488,#النهدي ثمانية قتلى في تفجير انتحاري بسيارة مف...,نهد ثما قتل تفجير انتحار سيار مفخخ معمل غاز ري...,549719610459967488
4,549720880717508608,البحرين: ضبط مطلوبين متورطين في التفجير بالعكر...,بحر ضبط مطلوب متورط تفجير عكر شرق بقي موضوع اضغط,549720880717508608
...,...,...,...,...
4995,551574113324920832,«بوكو حرام »تخطف 40 شابا في شمال شرق نيجيريا -...,وكو حرام تخطف 40 شاب نيجيري رياض رياض وكو حرام...,551574113324920832
4996,551574871797272576,قصيدة مهداة إلى الشيخ/ ضيف الله بن سمار كلمات/...,قصيد مهد شيخ ضيف الله سمار كلما سلطان حامد سما...,551574871797272576
4997,551574876092256256,العفو عند المقبرة!!,عفو مقبره,551574876092256256
4998,551575878141222912,#للمرة الأولى منذ 92 عاما.. تركيا تسمح ببناء ك...,مره اول 92 عام تركي تسمح بناء كنيس جديد مجل رساءل,551575878141222912


# Indexing

In [ ]:
indexer = pt.IterDictIndexer(
    "./arabic_index",
    overwrite=True,
    meta={"docno": 20, "text": 2048},  # Field lengths
    stemmer=None,      # Disable stemming (already done)
    stopwords=None,    # Disable stopwords (already removed)
    tokeniser="UTFTokeniser"  # Use UTF tokenizer for Arabic
)

docs = [{
    'docno': row['docno'],
    'text': row['processed_tweets']
} for _, row in tweets.iterrows()]

index_ref = indexer.index(docs)
index = pt.IndexFactory.of(index_ref)

print(index.getCollectionStatistics().toString())

Number of documents: 5000
Number of terms: 7194
Number of postings: 50955
Number of fields: 0
Number of tokens: 53484
Field names: []
Positions:   false



In [ ]:
!ls -lh myFirstIndex/

total 0


In [ ]:
print(index.getCollectionStatistics().toString())

Number of documents: 5000
Number of terms: 7194
Number of postings: 50955
Number of fields: 0
Number of tokens: 53484
Field names: []
Positions:   false



In [ ]:
lexicon = index.getLexicon()

for entry in lexicon:
    print(entry.getKey())

Streaming output truncated to the last 5000 lines.
تناغم
تناقش
تناول
تنب
تنباع
تنتخب
تنتزع
تنتقد
تنتقل
تنخفض
تندم
تنديد
تنزاني
تنزعج
تنزف
تنزل
تنس
تنسيق
تنشر
تنضم
تنضيم
تنطق
تنطيم
تنظم
تنظيم
تنفيذ
تنقل
تنكر
تنميه
تنهض
تهاب
تهدد
تهددفلسط
تهدم
تهديد
تهرطق
تهز
تهكر
تهكروم
تهكير
تهم
تو
توا
تواجد
توار
تواصل
تواضروس
تواضع
تواعد
توافد
توافق
توال
توت
توتر
توتنهام
توثيق
توج
توجد
توجه
توجوجك
توحيد
تور
تورط
توزيع
توصل
توصيل
توضيح
توظيف
توع
توعو
توف
توفيق
توق
توقع
توقعا
توقف
توقي
توقيت
توقيع
توقيف
تولع
تون
تونس
توه
توهق
توهير
تويتر
تويته
تيار
تيت
تيتم
تيسير
تيف
تيل
ثاءق
ثاب
ثاف
ثان
ثانو
ثانيه
ثر
ثقاف
ثقافه
ثقافي
ثقافيه
ثقب
ثقه
ثكنا
ثلاث
ثلاثاء
ثلووج
ثليماء
ثم
ثما
ثمار
ثنيه
ثوا
ثور
ثوره
ثوريه
ثيق
جا
جاء
جاب
جات
جاد
جار
جارد
جاره
جاز
جازان
جاسوس
جاك
جاليه
جامع
جامعت
جانب
جانبيه
جاهز
جاهل
جاي
جايه
جب
جبال
جبان
جباه
جبت
جبل
جبه
جبهه
جبور
جثث
جثمان
جثه
جثيم
جحش
جحيم
جد
جدا
جدال
جدالا
جدد
جدر
جدك
جدل
جده
جدول
جدوو
جدي
جديد
جديده
جدير
جراء
جراءم
جراح
جراف
جرب
جرت
جرح
جرد
جرذان
جرس
جرو
جروج
جروح
جرود
جروو

In [ ]:
#Retrieve Indexed Terms with Term Statistics
for x in index.getLexicon():
  print("%s : %s " % (x.getKey(), x.getValue().toString()))

Streaming output truncated to the last 5000 lines.
تناغم : term4727 Nt=1 TF=1 maxTF=1 @{0 22673 4} 
تناقش : term2552 Nt=2 TF=2 maxTF=1 @{0 22676 4} 
تناول : term5585 Nt=4 TF=4 maxTF=1 @{0 22681 6} 
تنب : term2537 Nt=1 TF=1 maxTF=1 @{0 22689 0} 
تنباع : term1896 Nt=1 TF=1 maxTF=1 @{0 22691 6} 
تنتخب : term3941 Nt=1 TF=1 maxTF=1 @{0 22694 2} 
تنتزع : term5970 Nt=1 TF=1 maxTF=1 @{0 22697 0} 
تنتقد : term3040 Nt=1 TF=1 maxTF=1 @{0 22700 0} 
تنتقل : term1315 Nt=3 TF=3 maxTF=1 @{0 22702 6} 
تنخفض : term5219 Nt=1 TF=1 maxTF=1 @{0 22709 2} 
تندم : term1787 Nt=1 TF=1 maxTF=1 @{0 22712 2} 
تنديد : term2321 Nt=2 TF=2 maxTF=1 @{0 22714 6} 
تنزاني : term5679 Nt=3 TF=3 maxTF=1 @{0 22719 0} 
تنزعج : term2773 Nt=2 TF=2 maxTF=1 @{0 22727 0} 
تنزف : term1286 Nt=1 TF=1 maxTF=1 @{0 22732 0} 
تنزل : term4133 Nt=2 TF=2 maxTF=1 @{0 22734 2} 
تنس : term2181 Nt=3 TF=3 maxTF=1 @{0 22740 2} 
تنسيق : term831 Nt=4 TF=4 maxTF=1 @{0 22747 0} 
تنشر : term2192 Nt=8 TF=8 maxTF=1 @{0 22754 6} 
تنضم : term837 Nt=73 TF=85

In [ ]:
#Index Structure/Properties:
with open("./arabic_index/data.properties", "r") as f:
    print(f.read())  # Display properties of the index

#/content/arabic_index/data_stream0_1.properties
#Sat May 10 21:53:20 UTC 2025
index.document-factory.parameter_values=
index.meta-inputstream.parameter_types=org.terrier.structures.IndexOnDisk,java.lang.String
index.lexicon.termids=fileinmem
index.meta-inputstream.class=org.terrier.structures.ZstdCompressedMetaIndex$InputStream
index.inverted-inputstream.class=org.terrier.structures.bit.BitPostingIndexInputStream
index.lexicon-valuefactory.class=org.terrier.structures.BasicLexiconEntry$Factory
index.document.class=org.terrier.structures.FSADocumentIndex
index.lexicon.class=org.terrier.structures.FSOMapFileLexicon
index.lexicon-inputstream.parameter_values=structureName,index
num.Documents=5000
index.lexicon-entry-inputstream.class=org.terrier.structures.FSOMapFileLexicon$MapFileLexiconEntryIterator
index.meta.class=org.terrier.structures.ZstdCompressedMetaIndex
index.inverted-inputstream.parameter_values=index,structureName,lexicon-entry-inputstream,org.terrier.structures.postings.bit

# **Query Processing:**

In [ ]:
#PREPROCESS Query
def search_term(term):

    term = preprocess(term)
    print(f"\n Searching for: {term}")

    try:
        pointer = index.getLexicon()[term]
        for posting in index.getInvertedIndex().getPostings(pointer):
            print(f"Document ID: {posting.getId()}, Length: {posting.getDocumentLength()}")
    except KeyError:
        print(f" Term '{term}' not found in the index.")

In [ ]:
query_1 = "يوتيوب"

In [ ]:
search_term(query_1)


 Searching for: يوتيوب
Document ID: 2192, Length: 15
Document ID: 2778, Length: 12
Document ID: 2779, Length: 12
Document ID: 2780, Length: 12
Document ID: 2781, Length: 12
Document ID: 2782, Length: 12
Document ID: 2783, Length: 12
Document ID: 2784, Length: 12
Document ID: 2785, Length: 12
Document ID: 2786, Length: 12


In [ ]:
query_2 = "هذا السجن"

In [ ]:
search_term(query_2)


 Searching for: سجن
Document ID: 3068, Length: 16
Document ID: 3107, Length: 20
Document ID: 4166, Length: 17
Document ID: 4217, Length: 7


In [ ]:
'''
tweet_content = tweets.loc[tweets["docno"] == "549679192804061184", "tweetText"].values

if len(tweet_content) > 0:
    print(tweet_content)  # Get the first match
else:
    print("No matching docno found.")
'''


['الاعدام لعامل مطعم قتل زميله طعناً في "البيادر" أيدت محكمة التمييز الحكم الصادر عن محكمة الجنايات الكبرى والقاضي... http://t.co/H0txdjv3Kn']


In [ ]:
# Get mapping between internal IDs and docnos
internal_ids = [2192, 2778, 2779, 2780, 2781, 2782, 2783, 2784, 2785, 2786]
for docid in internal_ids:
    docno = index.getMetaIndex().getItem("docno", docid)
    print(f"Internal ID {docid} → DocNo {docno}")

Internal ID 2192 → DocNo 550935857268785152
Internal ID 2778 → DocNo 551143688345100288
Internal ID 2779 → DocNo 551143692367441921
Internal ID 2780 → DocNo 551143693697048576
Internal ID 2781 → DocNo 551143696909873152
Internal ID 2782 → DocNo 551143700647014400
Internal ID 2783 → DocNo 551143705663373312
Internal ID 2784 → DocNo 551143708729430018
Internal ID 2785 → DocNo 551143713271869443
Internal ID 2786 → DocNo 551143721861779456


In [ ]:
# Query search using TF-IDF
tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)


<ipython-input-34-af12808ac98a>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)


In [ ]:
processed_query = preprocess(query_1)
tfidf_results = tfidf_retr.search(processed_query)
tfidf_results

,qid,docid,docno,rank,score,query
0,1,2778,551143688345100288,0,4.659758,يوتيوب
1,1,2779,551143692367441921,1,4.659758,يوتيوب
2,1,2780,551143693697048576,2,4.659758,يوتيوب
3,1,2781,551143696909873152,3,4.659758,يوتيوب
4,1,2782,551143700647014400,4,4.659758,يوتيوب
5,1,2783,551143705663373312,5,4.659758,يوتيوب
6,1,2784,551143708729430018,6,4.659758,يوتيوب
7,1,2785,551143713271869443,7,4.659758,يوتيوب
8,1,2786,551143721861779456,8,4.659758,يوتيوب
9,1,2192,550935857268785152,9,4.200683,يوتيوب


# **Query expansion**



PRF for query expansion

In [ ]:
# Define our retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=30)

<ipython-input-45-07d6ff4f755b>:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=30)


In [ ]:
original_query = "هجوم انتحاري على مقهى في طرابلس في لبنان"
query = preprocess(original_query)

In [ ]:
results = bm25.search(query)
results

,qid,docid,docno,rank,score,query
0,1,485,550280680823726080,0,11.130279,هجوم انتحار مقه طرابلس لبنان
1,1,252,550222464895442946,1,10.611605,هجوم انتحار مقه طرابلس لبنان
2,1,1482,550614493894811649,2,10.586036,هجوم انتحار مقه طرابلس لبنان
3,1,1494,550618947494481921,3,10.586036,هجوم انتحار مقه طرابلس لبنان
4,1,1502,550620873539862528,4,10.586036,هجوم انتحار مقه طرابلس لبنان
5,1,3682,551342519628615680,5,10.356927,هجوم انتحار مقه طرابلس لبنان
6,1,3715,551346601218682880,6,10.356927,هجوم انتحار مقه طرابلس لبنان
7,1,3755,551350611636412416,7,10.356927,هجوم انتحار مقه طرابلس لبنان
8,1,765,550359056540393473,8,10.092536,هجوم انتحار مقه طرابلس لبنان
9,1,1559,550635615406936065,9,10.092536,هجوم انتحار مقه طرابلس لبنان


In [ ]:
# "rewrite" from PyTerrier to expand queries specifying RM3 as the model
# fb_docs : no. expansion documents
# fb_terms: no. expansion terms

rm3_expander = pt.rewrite.RM3(index,fb_terms=4, fb_docs=10)

#output of the BM25 will be fed into the RM3 expander for query expansion.
rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]

expanded_query

'applypipeline:off طرابلس^0.120000005 كنيس^0.076505333 نيجيري^0.076505333 انتحار^0.120000005 لبنان^0.120000005 هجوم^0.286107719 اب^0.080881588'

In [ ]:
# Just print the expanded query with term scores
for s in expanded_query.split()[1:]:
  print(s)

print("\n" + query)

طرابلس^0.120000005
كنيس^0.076505333
نيجيري^0.076505333
انتحار^0.120000005
لبنان^0.120000005
هجوم^0.286107719
اب^0.080881588

هجوم انتحار مقه طرابلس لبنان


In [ ]:
expanded_query_formatted = ' '.join(expanded_query.split()[1:])

results_wqe = bm25.search(expanded_query_formatted)

print("   Before Expansion    After Expansion")
print(pd.concat([results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna(''))

#tweets text for the top 5 retrieved tweets
tweets[['tweetText']][tweets['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

   Before Expansion    After Expansion
   docid_1    score_1  docid_2    score_2
0      485  11.130279     1482  11.602993
1      252  10.611605     1494  11.602993
2     1482  10.586036     1502  11.602993
3     1494  10.586036     1569  10.569361
4     1502  10.586036     1570  10.569361


,tweetText
1482,http://t.co/fEaqgajaPb هجوم انتحاري أمام كنيسة...
1494,هجوم انتحاري أمام كنيسة إنجيلية في نيجيريا: ht...
1502,هجوم انتحاري أمام كنيسة إنجيلية في نيجيريا htt...
1569,انتحارية في هجوم فاشل وانتحاري يستهدف كنيسة بن...
1570,انتحارية في هجوم فاشل وانتحاري يستهدف كنيسة بن...
1578,إنتحارية في هجوم فاشل وإنتحاري يستهدف كنيسة بن...


Incorporate synonyms or related terms using pre-built mappings or Embeddings (RNN,
ELMo and BERT).

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load Arabic Sentence-BERT model
model = SentenceTransformer("CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment")

def extract_top_terms(docs, query, top_n=5):
    # Tokenize and remove duplicates
    candidate_terms = list(set(" ".join(docs).split()))

    # Get embeddings
    query_emb = model.encode(query, convert_to_tensor=True)
    term_embs = model.encode(candidate_terms, convert_to_tensor=True)

    # Compute similarity
    cos_scores = util.cos_sim(query_emb, term_embs)[0]
    top_indices = cos_scores.topk(top_n).indices

    top_terms = [candidate_terms[i] for i in top_indices]
    return top_terms


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Step 1: Use results after RM3-expanded BM25
rm3_docs = tweets[tweets['docno'].isin(results_wqe['docno'].loc[0:10].tolist())]
rm3_texts = rm3_docs['tweetText'].tolist()

top_bert_terms = extract_top_terms(rm3_texts, query, top_n=5)

bert_expanded_query_1 = query + " " + " ".join(top_bert_terms)

#Search
bert_results_1 = bm25.search(bert_expanded_query_1)


In [ ]:
bert_results_1

,qid,docid,docno,rank,score,query
0,1,1325,550568336958820352,0,14.397029,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
1,1,262,550224599254724609,1,11.311815,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
2,1,792,550364475246002176,2,11.311815,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
3,1,485,550280680823726080,3,11.130279,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
4,1,252,550222464895442946,4,10.611605,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
5,1,1482,550614493894811649,5,10.586036,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
6,1,1494,550618947494481921,6,10.586036,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
7,1,1502,550620873539862528,7,10.586036,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
8,1,1386,550587746507431936,8,10.444785,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...
9,1,1387,550587747384037377,9,10.444785,هجوم انتحار مقه طرابلس لبنان انتحاري انتحارية ...


In [ ]:
# Step 1: Use initial BM25 results
initial_docs = tweets[tweets['docno'].isin(results['docno'].loc[0:10].tolist())]
initial_texts = initial_docs['tweetText'].tolist()

# Step 2: Expand using BERT
top_bert_terms_2 = extract_top_terms(initial_texts, query, top_n=5)

# Step 3: Combine and search
bert_expanded_query_2 = query + " " + " ".join(top_bert_terms_2)
bert_results_2 = bm25.search(bert_expanded_query_2)

In [ ]:
print("   Original     BERT only    BERT After BM25 Expansion")
print(pd.concat([
    results[['docid','score']][0:5].add_suffix('_orig'),
    bert_results_1[['docid','score']][0:5].add_suffix('_bert'),
    bert_results_2[['docid','score']][0:5].add_suffix('_All')
], axis=1).fillna(''))

   Original     BERT only    BERT After BM25 Expansion
   docid_orig  score_orig  docid_bert  score_bert  docid_All  score_All
0         485   11.130279        1325   14.397029        252  15.573195
1         252   10.611605         262   11.311815       1325  14.397029
2        1482   10.586036         792   11.311815        262  11.311815
3        1494   10.586036         485   11.130279        792  11.311815
4        1502   10.586036         252   10.611605        485  11.130279


In [ ]:
def print_top_docs_with_original_ids(title, result_df, tweets_df, index):
    print(f"\n====== {title} ======\n")

    internal_ids = result_df['docid'].head(5).tolist()
    docnos = []
    for docid in internal_ids:
        try:
            docno = index.getMetaIndex().getItem("docno", docid)
            docnos.append(docno)
        except KeyError:
            print(f"Warning: Internal ID {docid} has no matching docno.")
            docnos.append(None)

    for docno in docnos:
        if docno is not None:
            # Retrieve
            doc_text = tweets_df[tweets_df['docno'] == docno]['tweetText'].values
            if doc_text.size > 0:
                print(f"DocNo: {docno}")
                print(f"Tweet: {doc_text[0]}")
                print("-" * 60)
            else:
                print(f"DocNo: {docno} - No matching text found!")
        else:
            print("No docno found for this internal ID.")


In [ ]:
print("query:", original_query)
print_top_docs_with_original_ids("Original BM25", results, tweets, index)
print_top_docs_with_original_ids("BERT Only Expansion", bert_results_2, tweets, index)
print_top_docs_with_original_ids("BERT After RM3 + BM25", bert_results_1, tweets, index)


query: هجوم انتحاري على مقهى في طرابلس في لبنان

====== Original BM25 ======

DocNo: 550280680823726080
Tweet: @akhbar الهجوم الانتحاري في مدينة اب
------------------------------------------------------------
DocNo: 550222464895442946
Tweet: الهجوم الانتحاري على مجلس النواب الذي تبناه داعش مدلول خطير جدا لفجوة امنية في اجهزة الدولة #ليبيا #لبنان_جديد
------------------------------------------------------------
DocNo: 550614493894811649
Tweet: http://t.co/fEaqgajaPb هجوم انتحاري أمام كنيسة إنجيلية في نيجيريا http://t.co/a30re6P2dP
------------------------------------------------------------
DocNo: 550618947494481921
Tweet: هجوم انتحاري أمام كنيسة إنجيلية في نيجيريا: http://t.co/YFjar8YYwy
------------------------------------------------------------
DocNo: 550620873539862528
Tweet: هجوم انتحاري أمام كنيسة إنجيلية في نيجيريا http://t.co/lHVXJETEfx
------------------------------------------------------------

====== BERT Only Expansion ======

DocNo: 550222464895442946
Tweet: الهجوم الانتح

## Same Code, just more organised way to be user friendly

In [ ]:
user_query = input("Enter your search query: ")
processed_query = preprocess(user_query)

In [ ]:
def get_bert_expanded_query(base_query, doc_df, text_column="tweetText", top_n=5):
    texts = doc_df[text_column].tolist()
    top_terms = extract_top_terms(texts, base_query, top_n)
    return base_query + " " + " ".join(top_terms)

In [ ]:
def search_query(query):
    processed_query = preprocess(query)
    bm25_results = bm25.search(processed_query)

    # RM3 Expansion
    rm3_expander = pt.rewrite.RM3(index, fb_terms=4, fb_docs=10)
    rm3_qe = bm25 >> rm3_expander
    expanded_query_rm3 = rm3_qe.search(processed_query).iloc[0]["query"]
    results_rm3 = bm25.search(expanded_query_rm3)

     #all expansion together
    expanded_query_bert1 = get_bert_expanded_query(processed_query, tweets[tweets['docno'].isin(bm25_results['docno'].head(10))])
    results_all = bm25.search(expanded_query_bert1)

    #bert only
    expanded_query_bert2 = get_bert_expanded_query(processed_query, tweets[tweets['docno'].isin(results_rm3['docno'].head(10))])
    results_bert = bm25.search(expanded_query_bert2)


    return bm25_results, results_rm3, results_all, results_bert


# Displaying Results For Evaluation
(comparing between models results)

In [ ]:
def display_results(title, results):
    print(f"\n{title}")
    for idx, row in results.iterrows():
        docno = index.getMetaIndex().getItem("docno", row['docid'])
        tweet_text = tweets[tweets['docno'] == docno]['tweetText'].values
        if tweet_text:
            print(f"DocNo: {docno}\nTweet: {tweet_text[0]}\n{'-'*60}")


In [ ]:
print ("Query", user_query)
bm25_results, results_rm3, result_all, results_bert = search_query(user_query)

display_results("BM25 Results", bm25_results)
display_results("RM3 Expanded Results", results_rm3)
display_results("BERT Expanded Results", results_bert)

Query اهلا وسهلا

BM25 Results
DocNo: 551441609670557696
Tweet: أبو أنس الليبي في حكم من تم القبض عليه ؟!ومن سهل للقوات الأمريكية لدخول #ليبيا #الثورة انهم العملااااءالخونه حكومة زيدان باتت مطلوبه للعدالة
------------------------------------------------------------
DocNo: 550384755171393537
Tweet: بعض المرضى " الفساد " لم يكفيهم التفجير اللي صار في طبرق والمعاناة اللي نعانوا فيها لا بل يتهمموا في أهل طبرق... http://t.co/zUqMQLtmu7
------------------------------------------------------------
DocNo: 550416810919878657
Tweet: “@klmet7aq: #كلمة_حق: عاجل: انفجار سيارة بجوار كنيسة بشارع 23 يوليو في بورسعيد - http://t.co/pdbHCc7wMp http://t.co/05OQUpneLS اهلاً بالعيد
------------------------------------------------------------
DocNo: 550655841787846656
Tweet: اللهم كن مع ضحايا التفجير الاجرامي في اب و ارزق اهلهم الصبر و السلوان و عوضهم خير مما فقدوا
------------------------------------------------------------
DocNo: 551313447947087873
Tweet: السجل | طالب بإحالته إلى محكمة الجنايات الدولية..: 

# **Report:**
https://docs.google.com/document/d/1b5iwZi5gd4BfBPHp1tJb4B-nmFKz9Mc0/edit?usp=sharing&ouid=104252837305589081512&rtpof=true&sd=true